In [1]:
import ray
import gymnasium as gym

In [2]:
ray.init()

2023-04-20 15:53:57,240	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.10
Ray version:,2.3.1


In [30]:
@ray.remote
class CustomEnv(gym.Wrapper):
    a = 1
    def __init__(self, env):
        super().__init__(env)
        self.env = env
    
    def step(self, action):
        super().step(action)

    def get_action_space(self):
        return ray.get(self.action_space)

In [5]:
env = gym.make("CartPole-v1")
env_obj = CustomEnv.remote(env)

In [6]:
state_obj = env.reset.remote()

In [13]:
ray.get(env)

ValueError: 'object_refs' must either be an ObjectRef or a list of ObjectRefs.

In [32]:
ee = gym.make("CartPole-v1", render_mode="human")

In [33]:
ee = CustomEnv.remote(ee)

In [6]:
ee.reset.remote()

ObjectRef(16310a0f0a45af5c460cac1346c532a06953dd0d0100000001000000)

In [7]:
ee.render.remote()

ObjectRef(c2668a65bda616c1460cac1346c532a06953dd0d0100000001000000)

In [38]:
ee.reset()

(array([0.01176013, 0.00756816, 0.00742697, 0.01727878], dtype=float32), {})

In [39]:
ee.render()

: 

In [10]:
ee.close.remote()

ObjectRef(32d950ec0ccf9d2a460cac1346c532a06953dd0d0100000001000000)

In [15]:
ee.a.remote()

AttributeError: 'ActorHandle' object has no attribute 'a'

In [36]:
ray.get(ee.get_action_space.remote())

RayTaskError(ValueError): [36mray::CustomEnv.get_action_space()[39m (pid=7668, ip=127.0.0.1, repr=<CustomEnv<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>>)
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 803, in ray._raylet.execute_task.function_executor
  File "c:\Users\tomkj\miniconda3\envs\rl\lib\site-packages\ray\_private\function_manager.py", line 674, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\tomkj\miniconda3\envs\rl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in _resume_span
    return method(self, *_args, **_kwargs)
  File "C:\Users\tomkj\AppData\Local\Temp\ipykernel_38488\153271184.py", line 12, in get_action_space
  File "c:\Users\tomkj\miniconda3\envs\rl\lib\site-packages\ray\_private\client_mode_hook.py", line 105, in wrapper
    return func(*args,